In [1]:
_ = """ Test Kappa:
program: block
block: '{' (expr';')+ return_expr? '}'

return_expr: '$' expr;

expr: ( object
       | binary_expr
       | func_app
       | if_expr
       | while_expr
       | (expr) )

token: literal | identifier | operator | syntax_symbol
object: literal | array | identifier | func_def

number: -?[0-9]+('.'[0-9]+)?
null: 'null'
bool: 'true' | 'false'
string: '"' ([^"]*) '"'
literal: number | null | bool
identifier: [_a-z]+ | identifier[literal]

operator: '+' | '-' | '/' | '*' | '**' | '%' | '//' | '=' | '==' | '<' | '>' | '<=' | '>=' | '!=' | '&&' | '||' | '!'
syntax_symbol: ';' | '(' | ')' | '{' | '}' | '[' | ']' | ',' | '$' | '?'

binary_expr: left=expr op=operator right=expr
array: [expr(','expr)+]
func_app: func_name=identifier '(' arg=list(expr) ')'
func_def: func_name=identifier '(' params=list(Identifier) ')' body=block
if_expr: '?' '(' cond=expr ')' iftrue=block
while_expr: '@' '(' cond=expr ')' body=block
"""

In [2]:
# Imports
from typing import Self, Callable, Type, Any
from collections.abc import Iterable
from operator import (
    add, sub, mul, floordiv, truediv, mod, pos, neg,
    eq, ne, gt, lt, ge, le, or_, and_, not_
)
from enum import Enum
from copy import deepcopy

In [3]:
# Exceptions
class ParsingException(Exception):
    pass

class InterpretationException(Exception):
    pass

class EnvLookupException(Exception):
    pass

class ReaderError(Exception):
    pass

class Returning(Exception):
    pass

In [4]:
# Base Classes
class Expr:
    def __repr__(self) -> str:
        return 'Expr'
    
class Return_Expr():
    def __init__(self, expr: Expr) -> None:
        self.expr = expr
    
class Token:
    def __repr__(self) -> str:
        return 'Token'
    
class Object(Expr):
    def __repr__(self) -> str:
        return 'Object'

In [5]:
# Literals
class Literal[T](Object, Token):
    def __init__(self, value: T) -> None:
        super().__init__()
        if not self.__class__.validate(value):
            raise TypeError(f'Value {value} of type {type(value)} cannot be stored as {self.__class__}')
        self.value = value

    @classmethod
    def validate(cls, value: T) -> bool:
        return True

    def __repr__(self) -> str:
        return f'{self.__class__.__name__}: {self.value.__repr__()}'
    
    def __str__(self) -> str:
        return str(self.value)
    
    def __eq__(self, other: Self) -> bool:
        if not isinstance(other, self.__class__):
            return False
        return self.value == other.value

class Number(Literal[int | float]):
    @classmethod
    def validate(cls, value: int | float) -> bool:
        return isinstance(value, int | float)

class String(Literal[str]):
    @classmethod
    def validate(cls, value: str) -> bool:
        return isinstance(value, str)

class Boolean(Literal[bool]):
    @classmethod
    def validate(cls, value: bool) -> bool:
        return isinstance(value, bool)

class Null(Literal[None]):
    def __init__(self, value: None = None) -> None:
        super().__init__(value)

    @classmethod
    def validate(cls, value: None) -> bool:
        return value is None

    def __repr__(self) -> str:
        return 'Null'
    
    def __str__(self) -> str:
        return 'Null'

In [6]:
# Identifier
class Identifier(Object, Token):
    def __init__(self, name: str) -> None:
        super().__init__()
        self.name = name

    def __repr__(self) -> str:
        return self.name
    
class Indexed_Identifier(Identifier):
    def __init__(self, name: str, index: Expr) -> None:
        super().__init__(name)
        self.index = index

In [7]:
# Operators
class Operator(Token):
    def __init__(self, name: str, func: callable, prec: int, return_type: Type[Object],
                 left_type: Type[Object], right_type: Type[Object]) -> None:
        super().__init__()
        self.name = name
        self.func = func
        self.prec = prec
        self.return_type = return_type
        self.operand_types = (left_type, right_type)

    def __call__(self, *args: Any, **kwds: Any) -> Any:
        return self.func.__call__(*args, **kwds)
    
    def __repr__(self) -> str:
        return self.name
    
class Unary_Operator(Operator):
    def __init__(self, name: str, func: callable, prec: int, return_type: type[Object], operand_type: type[Object]) -> None:
        super().__init__(name, func, prec, return_type, Null, operand_type)
    
class Arithmetic_Operator(Operator):
    def __init__(self, name: str, func: callable, prec: int) -> None:
        super().__init__(name, func, prec, Number, Number, Number)

class Comparison_Operator(Operator):
    def __init__(self, name: str, func: callable, prec: int) -> None:
        super().__init__(name, func, prec, Boolean, Number, Number)

class Plus(Operator):
    def __init__(self, prec: int) -> None:
        super().__init__('+', add, prec, Number, Number, Number)

class Minus(Operator):
    def __init__(self, prec: int) -> None:
        super().__init__('-', sub, prec, Number, Number, Number)

class Pos(Unary_Operator):
    def __init__(self, prec: int) -> None:
        super().__init__('+', pos, prec, Number, Number)

class Neg(Unary_Operator):
    def __init__(self, prec: int) -> None:
        super().__init__('-', neg, prec, Number, Number)

class SingleEqual(Operator):
    def __init__(self, prec: int) -> None:
        super().__init__('=', None, prec, Object, Identifier, Object)

In [8]:
# Syntax Symbols
class Syntax_Symbol(Token, Enum):
    open_parenthesis = '('
    close_parenthesis = ')'
    semicolon = ';'
    open_brace = '{'
    close_brace = '}'
    open_bracket = '['
    close_bracket = ']'
    comma = ','
    returner = '$'
    ifsymbol = '?'
    whilesymbol = '@'

    def __repr__(self) -> str:
        return f'<{self.value}>'
    
    def __str__(self) -> str:
        return f'<{self.value}>'

In [9]:
# Binary Expression
class Binary_Expr(Expr):
    def __init__(self, left: Expr, op: Operator, right: Expr) -> None:
        super().__init__()
        if not isinstance(op, Operator):
            raise TypeError(f'Binary_Expr cannot use non-operator \'{op}\'')
        self.left = left
        self.right = right
        self.op =op
        
    def __repr__(self) -> str:
        return f'({str(self.left) + ' ' if not isinstance(self.left, Null) else ''}{self.op} {self.right})'

In [10]:
# Arrays
class Array(list[Expr], Object):
    def __repr__(self) -> str:
        return super().__repr__()

In [11]:
# Functions
class Builtin_Function():
    def __init__(self, function: callable) -> None:
        self.function = function

    def __call__(self, *args: Any, **kwds: Any) -> Any:
        return self.function(*args, **kwds)
    
    def __repr__(self) -> str:
        return self.function.__repr__()
    
class Func_App(Expr):
    def __init__(self, func_name: Identifier, args: Iterable[Expr]) -> None:
        super().__init__()
        self.func_name = func_name
        self.args = args

    def __repr__(self) -> str:
        return f'{self.func_name}({self.args})'

In [12]:
# Environment
class Address():
    def __init__(self, env_name: str, loc: int) -> None:
        self.env_name = env_name
        self.loc =loc

    def __repr__(self) -> str:
        return f'Address: {self.loc}@{self.env_name}'

class Store(list[Expr]):
    def append(self, object: Expr) -> Address:
        super().append(object)
        return len(self) - 1

class Environment():
    def __init__(self, name: str, parent: Self = None) -> None:
        self.name = name
        self.parent = parent
        self.read_only = False

        self.store = Store()
        self.vars: dict[str, Address] = {}

    def alloc_var(self, var: Identifier) -> Address:
        if self.read_only:
            raise EnvLookupException(f'Cannot modify read_only {self}')
        if var.name in self.vars:
            raise EnvLookupException(f'{var} already exists in {self}')
        
        self.vars[var.name] = Address( self.name, self.store.append(Null()) )
        return self.vars[var.name]
    
    def locate_var(self, var: Identifier) -> Address:
        if var.name in self.vars:
            return self.vars[var.name]
        elif self.parent is not None:
            try:
                return self.parent.locate_var(var)
            except EnvLookupException as e:
                e.add_note(f'Call to {self.parent} from {self}')
                raise
        else:
            raise EnvLookupException(f'{self} could not find {var}')
        
    def get_var(self, addr: Address) -> Expr:
        if addr.env_name == self.name and addr.loc < len(self.store):
            return self.store[addr.loc]
        elif self.parent is not None:
            try:
                return self.parent.get_var(addr)
            except EnvLookupException as e:
                e.add_note(f'Call to {self.parent} from {self}')
                raise
        else:
            raise EnvLookupException(f'{self} could not locate {addr}')
        
    def set_var(self, addr: Address, expr: Expr) -> None:
        if addr.env_name == self.name and addr.loc < len(self.store):
            self.store[addr.loc] = expr
        elif self.parent is not None:
            try:
                self.parent.set_var(addr, expr)
            except EnvLookupException as e:
                e.add_note(f'Call to {self.parent} from {self}')
                raise
        else:
            raise EnvLookupException(f'{self} could not locate {addr}')
        
    def set_read_only(self) -> None:
        self.read_only = True

    def clear(self) -> None:
        self.vars = {}
        self.store = Store()

    def copy(self) -> Self:
        env = Environment(
            self.name,
            self.parent.copy() if self.parent is not None else None
        )
        env.read_only = self.read_only
        env.store = deepcopy(self.store)
        env.vars = deepcopy(self.vars)
        return env
        
    def __repr__(self) -> str:
        return 'Environment: ' + self.name

In [13]:
# Blocks
class Block:
    def __init__(self, name: str, env: Environment = None, expr_list: list[Expr] = None) -> None:
        self.name = name
        self.expr_list = [] if expr_list is None else expr_list
        self.env = env

    def add_expr(self, expr: Expr) -> None:
        self.expr_list.append(expr)

    def set_env(self, env: Environment) -> None:
        self.env = env

    def __repr__(self) -> str:
        return f'{{{self.expr_list}}}'
    
class Program(Block):
    def __init__(self, block: Block) -> None:
        super().__init__(block.name, block.env, block.expr_list)

In [14]:
# Function Block
class Func_Def(Object):
    def __init__(self, func_name: Identifier, params: Iterable[Identifier], body: Block) -> None:
        super().__init__()
        self.func_name = func_name
        self.body = body

        if not all(isinstance(x, Identifier) for x in params):
            raise ParsingException(f'Func_Def params must be Identifiers. Found fault paramaters {[x for x in params if not isinstance(x, Identifier)]}.')
        self.params = params
        
    def __repr__(self) -> str:
        return f'{self.func_name}({self.params}) {self.body}'


In [15]:
# If Block
class If_Expr(Expr):
    def __init__(self, cond: Expr, iftrue: Block) -> None:
        super().__init__()
        self.cond = cond
        self.iftrue = iftrue

    def __repr__(self) -> str:
        return f'?({self.cond}) {self.iftrue}'

In [16]:
# While Block
class While_Expr(Expr):
    def __init__(self, cond: Expr, body: Block) -> None:
        super().__init__()
        self.cond = cond
        self.body = body

    def __repr__(self) -> str:
        return f'@({self.cond}) {self.body}'

In [17]:
# Reader
class Reader[T]:
    # Note: get functions change cursor, peek functions do not change cursor
    def __init__(self, data: Iterable[T], cursor: int = 0) -> None:
        self.data = data
        self.length = len(self.data)
        self._cursor = cursor
        self.line = 0

    def isFinished(self) -> bool:
        return self._cursor >= self.length
    
    def next_line(self) -> None:
        self.line += 1
    
    def move(self, shift: int) -> None:
        if not (0 <= self._cursor + shift <= self.length):
            raise ReaderError(f'Reader move(): cannot move {self._cursor=} by {shift}')
        self._cursor += shift
    
    def get(self) -> T:
        if self.isFinished():
            raise ReaderError(f'Reader get(): No more elements in list.')

        self._cursor += 1
        return self.data[self._cursor - 1] # get the element present at cursor before increment
    
    def get_slice(self, size: int, start_shift: int = 0) -> Iterable[T]:
        start = self._cursor + start_shift
        stop = self._cursor + start_shift + size

        if not (0 <= start < self.length) or not (start <= stop <= self.length):
            error = ReaderError(f'Reader get_slice(): Cannot get sequence from {start} to {stop}')
            error.add_note(f'Current parameter values: {self.length, self._cursor, size, start_shift = }')
            raise error
        else:
            self._cursor = stop
            return self.data[start : stop]
        
    def get_until(self, ismatch: Callable[[T], bool]) -> Iterable[T]:
        try:
            count = 0
            while not ismatch(el := self.get()):
                count += 1
        except ReaderError:
            raise ReaderError(f'Reader get_until(): Target not found.')
        return self.get_slice(count, -(count+1))

    def peek(self) -> T:
        loc = self._cursor
        element = self.get()
        self._cursor = loc
        return element
    
    def peek_slice(self, size: int, start_shift: int = 0) -> Iterable[T]:
        loc = self._cursor
        elements = self.get_slice(size, start_shift)
        self._cursor = loc
        return elements
    
    def peek_nearby(self, side_size: int = 3, start_shift: int = 0) -> Iterable[T]:
        start_shift = max(-side_size, -self._cursor) # prevent going less than 0
        size = -start_shift + 1 + min(side_size, (self.length - 1) - self._cursor) # prevent going more than length
        return self.peek_slice(size, start_shift)

In [18]:
# Lexer
class Lexer:
    valid_numerics: str = '0123456789.'
    valid_alpha: str = 'abcdefghijklmnopqrstuvwxyz_'
    valid_keyword_literals: dict[str, Token] = {
        'null': Null(), 'true': Boolean(True), 'false': Boolean(False)
    }
    valid_operators: dict[str, Operator] = {
        '=': SingleEqual(0),

        '||': Operator('||', or_, 1, Boolean, Boolean, Boolean),
        '&&': Operator('&&', and_, 2, Boolean, Boolean, Boolean),
        '!': Unary_Operator('!', not_, 3, Boolean, Boolean),

        '==': Comparison_Operator('==', eq, 4), '!=': Comparison_Operator('!=', ne, 4),
        '>': Comparison_Operator('>', gt, 4), '>=': Comparison_Operator('>=', ge, 4),
        '<': Comparison_Operator('<', lt, 4), '<=': Comparison_Operator('<=', le, 4),

        '+': Plus(5), '-': Minus(5), 

        '*': Arithmetic_Operator('*', mul, 6), '/': Arithmetic_Operator('/', truediv, 6), 
        '%': Arithmetic_Operator('%', mod, 6), '//': Arithmetic_Operator('//', floordiv, 6), 

        '**': Arithmetic_Operator('**', pow, 7),

        # Binary plus and minus will be converted to unary pos and neg (during parsing)
        'POS': Pos(8), 'NEG': Neg(8)
    }

    @staticmethod
    def tokenise(raw_data: str) -> Iterable[Token]:
        token_list: list[Token] = []
        data = Reader[str](raw_data.lower(), cursor=0)
        while (token := Lexer.next_token(data)) is not None:
            token_list.append(token)
        return token_list
    
    @staticmethod
    def next_token(data: Reader[str]) -> Token | None:
        if data.isFinished():
            return None
        
        match data.get():
            case ' ' | '\n' | '\t':
                return Lexer.next_token(data)
            
            case sym if sym in Syntax_Symbol:
                return Syntax_Symbol(sym)
            
            case '"':
                value = ''
                try:
                    value += data.get_until(lambda x: x == '"')
                    data.get() # consume the end quote
                except ReaderError:
                    raise ParsingException(f'Lexer next_token(): Unterminated string found:\n... {data.peek_nearby(10)} ...')
                return String(value)

            case c if c in Lexer.valid_numerics:
                value = c
                try:
                    value += data.get_until(lambda x: x not in Lexer.valid_numerics)
                except ReaderError as e:
                    e.add_note(f'Lexer next_token(): Attempted to read number at\n... {data.peek_nearby()} ...')
                    raise e

                if '.' in value:
                    return Number(float(value))
                else:
                    return Number(int(value))
            
            case c if c in Lexer.valid_alpha:
                value = c
                try:
                    value += data.get_until(lambda x: x not in Lexer.valid_alpha)
                except ReaderError as e:
                    e.add_note(f'Lexer next_token(): Attempted to read identifier/keyword at\n... {data.peek_nearby()} ...')
                    raise e

                if value in Lexer.valid_keyword_literals:
                    return Lexer.valid_keyword_literals[value]
                else:
                    return Identifier(value)
                
            case _ if data.peek_slice(2, -1) in Lexer.valid_operators: # double-character operators
                return Lexer.valid_operators[data.get_slice(2, -1)]
            
            case op if op in Lexer.valid_operators: # single-character operators
                return Lexer.valid_operators[op]
            
            case x:
                raise ParsingException(f'Lexer next_token(): Unrecognised character found: {x} near ... {data.peek_nearby()} ...')

In [19]:
# Parser
class Parser:
    @staticmethod
    def parse_program(data: Iterable[Token]) -> Program:
        return Program(Parser.parse_block(Reader[Token](data), 'Main'))

    @staticmethod
    def parse_block(data: Reader[Token], name: str) -> Block:
        if data.isFinished():
            raise ParsingException(f'Parser parse_block(): No Tokens Found.')

        block = Block(
            name, Environment(name, None)
        )
        match data.get():
            case Syntax_Symbol.open_brace:
                try:
                    while data.peek() is not Syntax_Symbol.close_brace:
                        data.next_line()

                        # Block's return statement
                        if data.peek() is Syntax_Symbol.returner:
                            data.move(1)
                            block.add_expr(
                                Return_Expr(
                                    Parser.parse_expr(data)
                                )
                            )
                        else:
                            block.add_expr(
                                Parser.parse_expr(data)
                            )
                            
                    data.move(1) # skip the close_brace
                except ReaderError:
                    raise ParsingException(f'Parser parse_block(): Unterminated block found.')
                return block
            
            case _:
                raise ParsingException(f'Parser parse_block(): Could not locate starting brace\n... {data.peek_nearby()} ...')
    
    @staticmethod
    def parse_expr(data: Reader[Token], terminators: list[Syntax_Symbol] = None) -> Expr:
        if terminators is None:
            terminators = [Syntax_Symbol.semicolon]

        operator_stack: list[Operator] = []
        operand_stack: list[Expr] = []
        prev_is_operand: bool = False

        def combinable() -> bool:
            if len(operator_stack) == 0:
                return False
            
            min_operands: int = 0
            match operator_stack[-1]:
                case Unary_Operator():
                    min_operands = 1

                case Operator():
                    min_operands = 2

            return len(operand_stack) >= min_operands
        
        def combine_operands() -> None:
            op = operator_stack.pop()
            match op:
                case Unary_Operator():
                    right = operand_stack.pop()
                    operand_stack.append(
                        Binary_Expr(Null(), op, right)
                    )

                case Operator():
                    right, left = operand_stack.pop(), operand_stack.pop()
                    operand_stack.append(
                        Binary_Expr(left, op, right)
                    )

        while (chunk := Parser.parse_chunk(data, terminators=terminators)) is not None:
            match chunk:
                case Expr():
                    operand_stack.append(chunk)
                    prev_is_operand = True

                case Plus() | Minus() as op if not prev_is_operand: # Unary plus, minus
                    match op:
                        case Plus():
                            op = Lexer.valid_operators['POS']
                        case Minus():
                            op = Lexer.valid_operators['NEG']

                    while combinable() and operator_stack[-1].prec > op.prec: # > not >=, because unary is right associative
                        combine_operands()
                    operator_stack.append(op)
                    prev_is_operand = False

                case Operator() as op:
                    while combinable() and operator_stack[-1].prec >= op.prec:
                        combine_operands()
                    operator_stack.append(op)
                    prev_is_operand = False

                case _:
                    raise ParsingException(f'Parser parse_expr(): Unexpected chunk {chunk} at line {data.line}')
        
        while combinable():
            combine_operands()

        if len(operator_stack) > 0:
            raise ParsingException(
                f'Parser parse_expr(): Cannot handle excess operators {operator_stack} at line {data.line}'
            )
        elif len(operand_stack) > 1:
            raise ParsingException(
                f'Parser parse_expr(): Encountered abnormal operand count {operand_stack} at line {data.line}\nMaybe missing a terminator.'
            )

        if len(operand_stack) == 1:
            return operand_stack[0]
        else:
            return Null()
                   
    @staticmethod
    def parse_chunk(data: Reader[Token], terminators: list[Syntax_Symbol]) -> Expr | Operator | None:
        # Chunk can be an object, operator, parenthesised expression, func_app etc.
        if data.isFinished():
            raise ParsingException(f'Parser parse_chunk(): No Tokens Found.')
        
        match data.get():
            case Syntax_Symbol.open_parenthesis:
                return Parser.parse_expr(data, terminators=[Syntax_Symbol.close_parenthesis])

            case Syntax_Symbol.open_bracket:
                return Array(
                    Parser.parse_multiple_expr(data, terminators=[Syntax_Symbol.close_bracket], delimiters=[Syntax_Symbol.comma])
                )

            # Functions
            case Identifier() as value if data.peek() is Syntax_Symbol.open_parenthesis:
                data.move(1)
                args = Parser.parse_multiple_expr(data, terminators=[Syntax_Symbol.close_parenthesis], delimiters=[Syntax_Symbol.comma])

                if data.peek() is Syntax_Symbol.open_brace:
                    body = Parser.parse_block(data, value.name)
                    return Func_Def(value, args, body)
                return Func_App(value, args)
            
            # String or Array Indexing
            case Identifier(name=name) if data.peek() is Syntax_Symbol.open_bracket:
                data.move(1)
                index = Parser.parse_expr(data, terminators=[Syntax_Symbol.close_bracket])
                return Indexed_Identifier(name, index)
            
            case Syntax_Symbol.ifsymbol:
                if data.peek() is not Syntax_Symbol.open_parenthesis:
                    raise ParsingException(f'Parser parse_chunk(): For if statement, expected ( after ?. However, found {data.peek()}.')
                
                data.move(1)
                cond = Parser.parse_expr(data, terminators=[Syntax_Symbol.close_parenthesis])

                if data.peek() is not Syntax_Symbol.open_brace:
                    raise ParsingException(f'Parser parse_chunk(): if statement found without body. Expected {{ found {data.peek()}.')
                
                body = Parser.parse_block(data, '')
                return If_Expr(cond, body)
            
            case Syntax_Symbol.whilesymbol:
                if data.peek() is not Syntax_Symbol.open_parenthesis:
                    raise ParsingException(f'Parser parse_chunk(): For while statement, expected ( after @. However, found {data.peek()}.')
                
                data.move(1)
                cond = Parser.parse_expr(data, terminators=[Syntax_Symbol.close_parenthesis])

                if data.peek() is not Syntax_Symbol.open_brace:
                    raise ParsingException(f'Parser parse_chunk(): while statement found without body. Expected {{ found {data.peek()}.')
                
                body = Parser.parse_block(data, '')
                return While_Expr(cond, body)
                
            case Literal() | Identifier() | Operator() as value:
                return value
            
            case terminal if terminal in terminators:
                return None
            
            case x:
                e = ParsingException(
                    f'Parser parse_chunk(): Unexpected token {x} at line {data.line}'
                )
                e.add_note(f'Expected terminators: {', '.join([str(t) for t in terminators])}')
                raise e
            
    @staticmethod
    def parse_multiple_expr(data: Reader[Token], terminators: list[Syntax_Symbol], delimiters: list[Syntax_Symbol]) -> list[Expr]:
        expr_list: list[Expr] = []
        end = None
        while end not in terminators:
            expr = Parser.parse_expr(data, terminators=(terminators + delimiters))
            data.move(-1)
            end = data.get()

            if isinstance(expr, Null) and (end not in terminators):
                raise ParsingException(f'Parser parse_multiple_expr(): Encountered unexpected delimiter at line {data.line} in {data.peek_nearby()}')
            
            if not isinstance(expr, Null):
                expr_list.append(expr)
        return expr_list

In [20]:
# Interpeter
class Interpreter:
    @staticmethod
    def interp_prog(prog: Program, builtins: Environment) -> None:
        prog.set_env(
            Environment(prog.name, builtins)
        )
        try:
            Interpreter.interp_block(prog)
        except Returning:
            raise InterpretationException(f'Program returned from outside a function block.')

    @staticmethod
    def interp_func_block(block: Block) -> Literal:
        try:
            return Interpreter.interp_block(block)
        except Returning as e:
            block.env.clear()
            return e.args[0]

    @staticmethod
    def interp_block(block: Block) -> Literal:
        for expr in block.expr_list:
            _ = Interpreter.interp_expr(expr, block.env)
        block.env.clear()
        return Null()

    @staticmethod
    def interp_expr(expr: Expr, env: Environment) -> Literal:
        match expr:
            case Return_Expr():
                raise Returning( Interpreter.interp_expr(expr.expr, env) )

            case Literal():
                return expr
            
            case Indexed_Identifier():
                var = env.get_var( env.locate_var(expr) )

                index = Interpreter.interp_expr(expr.index, env)
                if not isinstance(index, Number) or index.value != int(index.value):
                    raise InterpretationException(f'Interpreter interp_expr(): {index} is not a valid index.')

                try: 
                    match var:
                        case Array():
                            return var[index.value]
                        case String():
                            return String(var.value[index.value])
                except IndexError:
                    raise IndexError(f'Interpreter interp_expr(): {index=} out of bounds for {var}.')
                    
                raise InterpretationException(f'Interpreter interp_expr(): Cannot index object of type {type(var)}')
            
            case Identifier():
                return env.get_var( env.locate_var(expr) )
            
            case Array():
                return Array(Interpreter.interp_multiple_expr(expr, env))
            
            case Func_App():
                func = env.get_var( env.locate_var(expr.func_name) )
                args = Interpreter.interp_multiple_expr(expr.args, env)
                return Interpreter.interp_func(func, args, env)
            
            case Func_Def():
                func = deepcopy(expr)
                env.set_var( env.alloc_var(func.func_name), func )
                func.body.set_env(
                    Environment(func.body.name, env.copy())
                )
                return Null()
            
            case If_Expr():
                cond = Interpreter.interp_expr(expr.cond, env)
                match cond:
                    case Boolean():
                        if cond.value:
                            block = expr.iftrue
                            block.set_env(
                                Environment('_if', env)
                            )
                            Interpreter.interp_block(block)
                        return Null()

                    case _:
                        raise InterpretationException(f'Interpreter interp_expr(): If statement condition must be a Boolean. Found {type(cond)}.')
                    
            case While_Expr():
                while True:
                    cond = Interpreter.interp_expr(expr.cond, env)
                    match cond:
                        case Boolean():
                            if cond.value:
                                block = expr.body
                                block.set_env(
                                    Environment('_while', env)
                                )
                                Interpreter.interp_block(block)
                            else:
                                return Null()

                        case _:
                            raise InterpretationException(f'Interpreter interp_expr(): While statement condition must be a Boolean. Found {type(cond)}.')
            
            case Binary_Expr():
                return Interpreter.interp_binary_expr(expr, env)
            
        raise InterpretationException(f'Interpreter interp_expr(): Cannot interpret expression {expr}.')
    
    @staticmethod
    def interp_func(func: Builtin_Function | Func_Def, args: list[Expr], env: Environment) -> Literal:
        match func:
            case Builtin_Function():
                return func(env, *args)
            
            case Func_Def():
                try:
                    for var, value in zip(func.params, args, strict=True):
                        try:
                            addr = func.body.env.locate_var(var)
                        except EnvLookupException:
                            addr = func.body.env.alloc_var(var)
                        func.body.env.set_var( addr, value )
                    return Interpreter.interp_func_block(func.body)
                except ValueError:
                    raise InterpretationException(f'Interpreter interp_func(): Required params {func.params} but received args {args}.')
                
            case _:
                raise InterpretationException(f'Interpreter interp_func(): {func} is not a function.')
            
    @staticmethod
    def interp_binary_expr(expr: Binary_Expr, env: Environment) -> Literal:
        match expr:
            case Binary_Expr(op = Unary_Operator()):
                op, r = expr.op, Interpreter.interp_expr(expr.right, env)
                if isinstance(r, op.operand_types[1]):
                    result = op.func(r.value)
                    return op.return_type(result)
                raise InterpretationException(f'Interpreter interp_binary_expr(): Unary {op} cannot operate on {r}')
            
            case Binary_Expr(left = Indexed_Identifier(), op = SingleEqual()):
                l, op, r = expr.left, expr.op, Interpreter.interp_expr(expr.right, env)
                if isinstance(r, op.operand_types[1]):
                    addr = env.locate_var(l)
                    var = env.get_var(addr)
                    index = Interpreter.interp_expr(l.index, env)
                    if not isinstance(index, Number) or index.value != int(index.value):
                        raise InterpretationException(f'Interpreter interp_expr(): {index} is not a valid index.')

                    try: 
                        match var:
                            case Array():
                                var[index.value] = r
                                env.set_var(addr, var)
                                return r
                            case String():
                                raise InterpretationException(f'Interpreter interp_expr(): String is not mutable.')
                    except IndexError:
                        raise IndexError(f'Interpreter interp_expr(): {index=} out of bounds for {var}.')
                        
                    raise InterpretationException(f'Interpreter interp_expr(): Cannot index object of type {type(var)}')

            case Binary_Expr(op = SingleEqual()):
                l, op, r = expr.left, expr.op, Interpreter.interp_expr(expr.right, env)
                if (
                    isinstance(l, op.operand_types[0]) and
                    isinstance(r, op.operand_types[1])
                ):
                    try:
                        addr = env.locate_var(l)
                    except EnvLookupException:
                        addr = env.alloc_var(l)
                    env.set_var(addr, r)
                    return r
                raise InterpretationException(f'Interpreter interp_binary_expr(): Cannot assign {r} to {l}')
            
            case Binary_Expr():
                l, op, r = Interpreter.interp_expr(expr.left, env), expr.op, Interpreter.interp_expr(expr.right, env)
                if (
                    isinstance(l, op.operand_types[0]) and
                    isinstance(r, op.operand_types[1])
                ):
                    result = op.func(l.value, r.value)
                    return op.return_type(result)
                raise InterpretationException(f'Interpreter interp_binary_expr(): {op} cannot operate on {l, r}')
        raise InterpretationException(f'Interpreter interp_binary_expr(): Cannot interpret non-binary expression {expr}.')
            
    @staticmethod
    def interp_multiple_expr(expr_list: list[Expr], env: Environment) -> Literal:
        return list(map(
            lambda el: Interpreter.interp_expr(el, env), expr_list
        ))

In [21]:
# Builtins
def printing_func(_: Environment, arg: Expr, end_line: String = None) -> Null:
    match arg, end_line:
        case Expr(), String():
            print(arg, end=end_line.value)
            return Null()
        
        case Expr(), None:
            print(arg)
            return Null()
        
        case _:
            raise InterpretationException(f'put() cannot handle following arguments: {arg}. Expected Expr or Expr, String.')

BUILTIN_ENV = Environment('Builtin')
for var, expr in [
    (Identifier('pi'), Number(3.14)), (Identifier('put'), Builtin_Function(printing_func))
]:
    BUILTIN_ENV.set_var( BUILTIN_ENV.alloc_var(var), expr )
BUILTIN_ENV.set_read_only()

In [23]:
# Testing
codeA = """{
y = 3 * pi; 
x = 4;
12+4;
put(!(pi * y != 25 && 3 <= 4) || --+-2 == -----2);
put(y / pi);
arr = 3;
arr = ([3+x**2+3, 1932.3 - 1, 199.2, x, 3 ** 2 ** 3]);
arr[0] = 1;
f = "false";
put(f[-2]);
[8+-( 2.0-1*9)];
}"""

codeB = """{
f(x) {
    z = 2;
    g(y) {
        put(x * y + z);
    };
    $g;
};
double_plus_two = f(2);
double_plus_two(3);
triple_plus_two = f(3);
triple_plus_two(3);
double_plus_two(3);
}"""

codeC = """
{
    rec_factorial(self, n) {
        ?(n < 1 || n != n // 1) {
            $Null;
        };

        ?(n == 1) {
            $1;
        };

        $n * self(self, n - 1);
    };

    factorial(n) {
        $rec_factorial(rec_factorial, n);
    };

    n = 0;
    @(n < 10) {
        put(n, "! = ");
        put(factorial(n));
        n = n + 1;
    };
    put("", "pi! = ");
    put(factorial(pi));
}
"""

codeD = """
{
    add(a, b, c) {
        $ a + b + c;
    };
    put(add(3, 4, 5));
}
"""

raw_code = codeC
lexed_code = Lexer.tokenise(raw_code)
prog = Parser.parse_program(lexed_code)
Interpreter.interp_prog(prog, BUILTIN_ENV)

0! = Null
1! = 1
2! = 2
3! = 6
4! = 24
5! = 120
6! = 720
7! = 5040
8! = 40320
9! = 362880
pi! = Null
